Note: some of the starting code was built upon notebooks developed for the FMA dataset, found at the FMA dataset github: https://github.com/mdeff/fma


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

from sklearn import metrics
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)#

In [2]:
features = pd.read_csv("fma_metadata/features.csv", header=[0,1,2], index_col=0)
tracks = pd.read_csv("fma_metadata/tracks.csv", header = [0,1], index_col=0)

# echonest = pd.read_csv("fma_metadata/echonest.csv", header = [0,1,2], index_col=0)
# genres = pd.read_csv("fma_metadata/genres.csv", index_col=0)

In [3]:
features.shape, tracks.shape #, echonest.shape, genres.shape

((106574, 518), (106574, 52))

In [4]:
features.sample(10)

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
134518       -0.063979 -0.203355 -0.745371 -0.749945  0.157842  0.285808   
31257         0.332571  0.054548  0.126548  0.862482  1.359049  1.181311   
110469       -0.237195 -0.826534 -0.708403  1.059986  1.871919  2.770892   
132149       -0.018203  0.509255  0.549885  0.698173 -0.194312  0.417158   
83653        -0.187374 -0.927695 -0.756921 -0.428983 -0.792085 -0.866159   
95137        -0.635754  0.283691 -0.330850 -0.927333 -0.390935 -0.023522   
133777        0.338817  0.059255  0.467413  0.036053 -0.793433  0.015231   
35203        -0.788852 -0.893618 -0.360461 -1.151021 -0.738055 -1.167874   
140282        0.112495 -0.494144 -0.132228 -0.232139  0.107320 -0.018270   
148242        0.061237 -0.138153 -0.722325  0.023461  0.252301  0.740661   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
134518     -0.046830  0.165702 -0.245630 -0.115655 -0.120910  0.133638   
31257       2.146429  0.629292  0.574945  0.019058  0.487625  0.535066   
110469      3.012524  1.178473 -0.235499  0.321599  1.073963  2.734535   
132149      0.090722  0.313226  2.095953 -0.000315 -0.458881  0.510137   
83653      -1.238725 -0.948540 -0.112915 -0.921431 -1.047466  0.680831   
95137       0.415243 -0.622423 -0.580161 -0.439219 -0.785843 -0.783957   
133777     -1.055483 -0.870687 -0.817795 -0.273317  0.585649 -0.524444   
35203      -1.508781 -0.134038 -0.769629 -0.403717 -0.453479 -0.168959   
140282      0.132544 -0.324307 -0.450877 -0.255303 -0.215387  0.858283   
148242     -0.080811 -0.420008  0.001837 -0.020793  0.112359 -0.429748   

feature                                                                 \
statistics       max                                                     
number            01        02        03        04        05        06   
track_id                                                                 
134518      0.709471  0.622665  0.709018  0.653372  0.684801  0.678242   
31257       0.660182  0.620274  0.686114  0.684689  0.691161  0.640316   
110469      0.648130  0.597270  0.558324  0.526476  0.488728  0.475565   
132149      0.720768  0.673963  0.701853  0.666663  0.628032  0.716872   
83653       0.643249  0.644627  0.697263  0.607210  0.689757  0.512930   
95137       0.527831  0.638377  0.639602  0.705105  0.611672  0.704831   
133777      0.666829  0.656874  0.590182  0.567515  0.690445  0.489960   
35203       0.550758  0.612001  0.632017  0.603667  0.664848  0.602536   
140282      0.477745  0.560674  0.583575  0.635067  0.467941  0.483898   
148242      0.587089  0.659994  0.655902  0.580115  0.634725  0.610894   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
134518      0.587671  0.668951  0.635805  0.671371  0.688026  0.642213   
31257       0.692397  0.635630  0.607029  0.688772  0.660419  0.548148   
110469      0.501061  0.489158  0.472676  0.447130  0.461537  0.673401   
132149      0.693587  0.685241  0.721941  0.639000  0.639446  0.723141   
83653       0.636849  0.681869  0.667565  0.682754  0.601632  0.600380   
95137       0.701719  0.589590  0.512381  0.638155  0.690840  0.535253   
133777      0.684511  0.457985  0.494761  0.412416  0.476517  0.693850   
35203       0.699320  0.649635  0.547738  0.592

In [5]:
tracks.sample(10)

album                                                        \
         comments         date_created        date_released    engineer   
track_id                                                                  
97239           0  2014-01-03 07:24:55  2014-01-03 00:00:00         NaN   
146910          0  2016-11-23 06:18:18  2015-01-21 00:00:00         NaN   
97614           0  2014-01-10 13:47:51  2014-01-10 00:00:00  Dan Barker   
127187          0  2015-11-03 07:36:42  2012-02-28 00:00:00         NaN   
113244          0  2015-01-06 22:44:36  2014-12-12 00:00:00         NaN   
15391           0  2009-06-23 02:47:07  2007-03-05 00:00:00  Thiaz Itch   
134404          6  2016-03-21 19:54:50  2016-03-22 00:00:00         NaN   
11090          -1                  NaN                  NaN         NaN   
119941          0  2015-05-20 14:58:48                  NaN         NaN   
36405           0  2010-09-28 06:52:37                  NaN         NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
97239            0  15771  <p>Здесь явно что-то не так, знакомые звуки сл...   
146910           0  21897  <p style="margin: 0px; font-size: 12px; line-h...   
97614            1  15812  <p>Renowned guitarist Gary Lucas is a former m...   
127187           1  19225  <p>Le groupe canadien <a href="http://www.wint...   
113244           1  17427  <p>Old school beats, breaks, and electro</p>\n...   
15391            2   3539  <p>This special Thiaz Itch album is a collecti...   
134404          28  20183  <p style="margin:0px 0px 8px;padding:0px;font-...   
11090           -1     -1                                                NaN   
119941           1  18304                                                NaN   
36405            3   7432  <p>"I need y'all to do me a favour again and g...   

                             \
         listens   producer   
track_id                      
97239       5264        NaN   
146910     34854        NaN   
97614      30154  Dan Bodah   
127187     14425        NaN   
113244     23116        NaN   
15391       4927        NaN   
134404    468967        NaN   
11090         -1        NaN   
119941     26211        NaN   
36405       5983        NaN   

                                                             \
                                                       tags   
track_id                                                      
97239                                                    []   
146910    ['improvisation', 'all styles experimental', '...   
97614                                                    []   
127187                                                   []   
113244    ['party', 'hip hop', 'funky', 'electronic', 'd...   
15391                     ['horror', 'ghost', 'electronic']   
134404                                                   []   
11090                                                    []   
119941                                                   []   
36405                                                    []   

                                                                           \
                                                      title tracks   type   
track_id                                                                    
97239                                              Rastanak      7  Album   
146910                                     Necktar Volume 7    148  Album   
97614     Live on WFMU's Airborne Event with Dan Bodah -...      9  Album   
127187                                               Oceans     10  Album   
113244                                    Rave New World EP      5  Album   
15391                     El baile de los fantasmas [WM059]      9  Album   
134404                           Music for TV & Film Vol. 1     42  Album   
1

# EDA / Data Cleaning

### Questions:
- What are the top genres in this dataset? 
- What does the distribution of top genres look like? 
- What to do with the huge class imbalance (counts differenct a factor of 100 - 1000)?

### Observations:
- The "tracks" dataset is organized with a top category (ie album, artist, set, track) and subcategories, so columns must be accessed in this way (see next cell as an example)
- Over HALF of the tracks do not have value in "genre_top" column (b/c they have multiple genres)
- Half of the tracks have multiple genres - how can these be used to classify songs?

#### the full dataset has NaN for about half of the genre_top feature

In [6]:
# Top Genres list for the entire set 
tracks.loc[:,('track','genre_top')].value_counts(dropna=False)

NaN                    56976
Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

In [7]:
# Create a large subset that drops rows with NaN in the genre_top column
large = tracks[tracks['track','genre_top'].notna()]
large.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

#### Notice there's a huge class imbalance in the largest dataset. I trim it down below:

In [8]:
# create a large subset with only genres that have at least 1000 tracks 
over_1000 = ["Rock","Experimental","Electronic","Hip_Hop","Folk","Pop","Instrumental","International","Classical"]
large_b = large[large['track','genre_top'].isin(over_1000)]
large_b.loc[:,('track','genre_top')].value_counts(dropna=False)

Rock             14182
Experimental     10608
Electronic        9372
Folk              2803
Pop               2332
Instrumental      2079
International     1389
Classical         1230
Name: (track, genre_top), dtype: int64

#### Grouping features to be used with the classification models

In [9]:
# What are the major features in the "features" dataset?
feature_list = features.columns.levels[0].tolist()
feature_list

['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr']

In [10]:
# Add a few feature COMBINATIONS (based on EDA done in the MVP)

# Top 2 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast'])
# Top 4 features with best accuracy scores by SVC classification
feature_list.append(['mfcc','spectral_contrast','chroma_cqt','chroma_stft'])
# All the features, EXCLUDING the 4 features with the worst accuracy scores by SVC classification
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','spectral_centroid',
                     'spectral_contrast','tonnetz'])
# All the features!
feature_list.append(['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr'])

# Indexes for the tables
indexes = ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
           'spectral_contrast','spectral_rolloff','tonnetz','zcr',
           'mfcc/spectral_contrast',
           'mfcc/spectral_contrast/chroma_cqt/chroma_stft',
           'chroma_cens/chroma_cqt/chroma_stft/mfcc/spectral_centroid/spectral_contrast/tonnetz',
           'ALL']

In [11]:
feature_list


['chroma_cens',
 'chroma_cqt',
 'chroma_stft',
 'mfcc',
 'rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_contrast',
 'spectral_rolloff',
 'tonnetz',
 'zcr',
 ['mfcc', 'spectral_contrast'],
 ['mfcc', 'spectral_contrast', 'chroma_cqt', 'chroma_stft'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'spectral_centroid',
  'spectral_contrast',
  'tonnetz'],
 ['chroma_cens',
  'chroma_cqt',
  'chroma_stft',
  'mfcc',
  'rmse',
  'spectral_bandwidth',
  'spectral_centroid',
  'spectral_contrast',
  'spectral_rolloff',
  'tonnetz',
  'zcr']]

# Data Analysis


In [12]:
# for lack of a better method, create a mask, large_b, that only has "top_genres" with at least 1000tracks
large_b = ((tracks['track','genre_top'].notna()) & (tracks['track','genre_top'].isin(over_1000)))
large_b

track_id
2         False
3         False
5         False
10         True
20        False
          ...  
155316     True
155317     True
155318     True
155319     True
155320    False
Name: (track, genre_top), Length: 106574, dtype: bool

In [13]:
accuracy_scores = pd.DataFrame(index=indexes)

# add first column that shows the number of dimensions for each feature (or collection of features)
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
accuracy_scores['dim']=dim_col

accuracy_scores

,dim
chroma_cens,84
chroma_cqt,84
chroma_stft,84
mfcc,140
rmse,7
spectral_bandwidth,7
spectral_centroid,7
spectral_contrast,49
spectral_rolloff,7
tonnetz,42


In [14]:
recall_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
recall_scores['dim']=dim_col


In [15]:
f1_scores = pd.DataFrame(index=indexes)

# Add first column with the number of dimensions for each of the features
dim_col = []
for feature in feature_list:
    X = features.loc[large_b, (feature)]
    dim_col.append(X.shape[1])
f1_scores['dim']=dim_col


### Logistic Regression

In [16]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['lr'] = accuracy_col
recall_scores['lr'] = recall_col
# f1_scores['lr'] = f1_col

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

ValueError: Length of values (0) does not match length of index (15)

In [ ]:
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

def train_and_save_model(feature_list, tracks, features, large_b, model_name):
    accuracy_col = []
    recall_col = []
    f1_col = []

    for feature in feature_list:
        y = tracks.loc[large_b, ('track','genre_top')]
        X = features.loc[large_b, feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        scaler = StandardScaler(copy=False)
        scaler.fit_transform(X_train)
        scaler.transform(X_test)
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
        recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
        f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

    results = {'accuracy': accuracy_col, 'recall': recall_col, 'f1': f1_col}

    with open(f'{model_name}.pickle', 'wb') as f:
        pickle.dump(results, f)

    return results


#### Weighted Logistic Regression

In [17]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr_weighted = LogisticRegression(max_iter=1000,class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
    lr_weighted.fit(X_train, y_train)
    y_pred = lr_weighted.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted lr'] = accuracy_col
# recall_scores['weighted lr'] = recall_col
# f1_scores['weighted lr'] = f1_col

CPU times: user 22min 23s, sys: 3.86 s, total: 22min 27s
Wall time: 22min 16s


### K Nearest Neighbors

In [18]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, (feature)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=200)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['knn'] = accuracy_col
# recall_scores['knn'] = recall_col
# f1_scores['knn'] = f1_col

CPU times: user 1min 58s, sys: 20.6 s, total: 2min 19s
Wall time: 1min 11s


### Decision Tree

In [19]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    DT = DecisionTreeClassifier(max_depth=5)
    DT.fit(X_train, y_train)
    y_pred = DT.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['dt'] = accuracy_col
# recall_scores['dt'] = recall_col
# f1_scores['dt'] = f1_col

CPU times: user 36.5 s, sys: 1.62 s, total: 38.1 s
Wall time: 38.1 s


In [ ]:
'''
Im aiming to create a function here so that the code is not so repetitive, not sure the best way to do so....
'''
'''
models = ['DecisionTreeClassifier(max_depth=5)',]
def test(analysis):
    for test_type in analysis:
        for feature in feature_list:
            y = tracks.loc[large_b, ('track','genre_top')]
            X = features.loc[large_b, feature]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            scaler = StandardScaler(copy=False)
            scaler.fit_transform(X_train)
            scaler.transform(X_test)
            test_type.fit(X_train, y_train)
            y_pred = test_type.predict(X_test)
            accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))
'''

### Random Forest

In [20]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    RF = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['rf'] = accuracy_col
# recall_scores['rf'] = recall_col
# f1_scores['rf'] = f1_col

CPU times: user 6.36 s, sys: 1.66 s, total: 8.02 s
Wall time: 8.03 s


### Ada Boost

In [21]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    ada = AdaBoostClassifier(n_estimators=10)
    ada.fit(X_train, y_train)
    y_pred = ada.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['ada'] = accuracy_col
# recall_scores['ada'] = recall_col
# f1_scores['ada'] = f1_col

CPU times: user 1min 21s, sys: 2.05 s, total: 1min 23s
Wall time: 1min 23s


### Naive Bayes

In [22]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    nbc_base = GaussianNB()
    nbc_base.fit(X_train, y_train)
    y_pred = nbc_base.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['nb'] = accuracy_col
# recall_scores['nb'] = recall_col
# f1_scores['nb'] = f1_col

CPU times: user 5.86 s, sys: 2.28 s, total: 8.13 s
Wall time: 8.16 s


### SVC Classifier

In [23]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

CPU times: user 58min 23s, sys: 39 s, total: 59min 2s
Wall time: 1h 31min 48s


In [24]:
accuracy_scores

,dim,lr,weighted lr,knn,dt,rf,ada,nb,svc
chroma_cens,84,0.440,0.426,0.434,0.391,0.358,0.349,0.179,0.519
chroma_cqt,84,0.433,0.422,0.435,0.390,0.370,0.367,0.046,0.528
chroma_stft,84,0.463,0.454,0.467,0.437,0.422,0.386,0.081,0.527
mfcc,140,0.606,0.595,0.571,0.514,0.484,0.487,0.447,0.672
rmse,7,0.440,0.431,0.457,0.442,0.451,0.433,0.247,0.460
spectral_bandwidth,7,0.408,0.401,0.457,0.443,0.446,0.423,0.399,0.464
spectral_centroid,7,0.417,0.409,0.466,0.446,0.446,0.437,0.392,0.467
spectral_contrast,49,0.553,0.547,0.527,0.477,0.473,0.456,0.444,0.626
spectral_rolloff,7,0.425,0.416,0.466,0.461,0.453,0.440,0.393,0.474
tonnetz,42,0.454,0.444,0.447,0.414,0.393,0.390,0.324,0.513


In [25]:
accuracy_scores.to_excel("accuracyScores.xlsx")

#### Weighted SVC classifier

In [28]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(kernel='rbf',class_weight={'International' : 4, 'Classical' : 4, 'Instrumental':3,'Folk':2, 'Pop':2})
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['weighted svc'] = accuracy_col
# recall_scores['weighted svc'] = recall_col
# f1_scores['weighted svc'] = f1_col

CPU times: user 1h 5min 3s, sys: 46.5 s, total: 1h 5min 49s
Wall time: 1h 6min 3s


#### SVC optimized with grid search

In [ ]:
%%time

accuracy_col = []
recall_col = []
f1_col = []

for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    SVCrbf = SVC(C=10,kernel='rbf')
    SVCrbf.fit(X_train, y_train)
    y_pred = SVCrbf.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))
#     recall_col.append(metrics.recall_score(y_test, y_pred,average='macro').round(3))
#     f1_col.append(metrics.f1_score(y_test, y_pred,average='macro').round(3))

accuracy_scores['svc'] = accuracy_col
# recall_scores['svc'] = recall_col
# f1_scores['svc'] = f1_col

In [ ]:
accuracy_scores

#### Optimize SVC with grid search

In [ ]:
# I couldn't fully run this. Took too many hours.

y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, 'mfcc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)


# defining parameter range 
param_grid = {'C': [0.01,0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'gamma': ['scale', 'auto'],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 

# fitting the model for grid search 
grid.fit(X_train, y_train)

# print best parameter after tuning 
print(grid.best_params_) 
y_pred = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_scores.to_excel("accuracy_scores.xlsx")

# Visualizations

## Confusion Matrix

In [ ]:
# Logistic Regression with all of the features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
sorted(tracks.loc[large_b,('track','genre_top')].unique())

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Logistic Regression with weighted genres and all features
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

lr_weighted = LogisticRegression(class_weight={'International' : 4, 'Classical' : 4, 
                                               'Instrumental':3,'Folk':2, 'Pop':2}, solver='liblinear')
lr_weighted.fit(X_train, y_train)
y_pred = lr_weighted.predict(X_test)
print("weighted lr accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("weighted lr recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("weighted lr f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))


In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0);

In [ ]:
# Confusion matrix with SVM classifier, all genres
y = tracks.loc[large_b, ('track','genre_top')]
X = features.loc[large_b, ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                           'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)
SVCrbf = SVC(kernel='rbf')
SVCrbf.fit(X_train, y_train)
y_pred = SVCrbf.predict(X_test)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred).round(3))
print("recall score: ", metrics.recall_score(y_test, y_pred,average='macro').round(3))
print("f1 score: ", metrics.f1_score(y_test, y_pred,average='macro').round(3))
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
plt.figure(dpi=150)
sns.set(font_scale=0.5)
sns.heatmap(confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt = "d",
           xticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'],
                        
           yticklabels=['Classical','Electronic','Experimental','Folk',
                        'Instrumental','International','Pop','Rock'])
plt.xlabel('Predicted Genre')
plt.ylabel('Actual Genre')
plt.title('Confusion Matrix')
plt.yticks(rotation=0)
plt.savefig('svc confusion_matrix'+'.jpg');

## PCA plots

In [ ]:
# Two genres that are RARELY misclassified as each other
# Note: this code was loosely copied from the FMA "usage" notebook (https://nbviewer.org/github/mdeff/fma/blob/outputs/usage.ipynb)
# all starting materials found at their github: https://github.com/mdeff/fma

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'spectral_contrast']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical spectral contrast', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Folk'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Folk vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Classical'

X = features.loc[large_b & (genre1 | genre2), ['chroma_cens','chroma_cqt','chroma_stft','mfcc','rmse','spectral_bandwidth','spectral_centroid',
                     'spectral_contrast','spectral_rolloff','tonnetz','zcr']]
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Rock vs Classical all features', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are RARELY misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Classical'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('PCA Electronic vs Classical', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are SOMETIMES misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Electronic'
genre2 = tracks['track', 'genre_top'] == 'Folk'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Folk', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=2)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Rock'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Rock', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Experimental'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Electronic vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='seismic', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Experimental'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Experimental', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape


In [ ]:
# Two genres that are OFTEN misclassified as each other

genre1 = tracks['track', 'genre_top'] == 'Pop'
genre2 = tracks['track', 'genre_top'] == 'Electronic'

X = features.loc[large_b & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[large_b & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.figure(figsize=(10,10))
plt.title('Pop vs Electronic', fontsize=24)
plt.xlabel('PC1', fontsize=16)
plt.ylabel('PC2', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.5, s=1)
X.shape, y.shape


## Tensorflow

In [29]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt



ModuleNotFoundError: No module named 'tensorflow'

In [30]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = (X_train,y_train),(X_test,y_test)

NameError: name 'tf' is not defined

In [31]:
for feature in feature_list:
    y = tracks.loc[large_b, ('track','genre_top')]
    X = features.loc[large_b, feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_test)
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    accuracy_col.append(metrics.accuracy_score(y_test, y_pred).round(3))


KeyboardInterrupt: 